In [17]:
import pandas as pd
import re
import math
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

In [18]:
packets = pd.read_csv("packets_0.csv")
packets = packets[packets["mac"] == "48:a3:80:0b:7c:fa"]

In [19]:
node_groups = packets.groupby("node_id")
node1_packets = node_groups.get_group(1).drop_duplicates()
node2_packets = node_groups.get_group(2).drop_duplicates()
node3_packets = node_groups.get_group(3).drop_duplicates()


In [26]:
merged_data = node1_packets.merge(node2_packets, on="ts").merge(node3_packets, on="ts").filter(
    ["rssi_x", "rssi_y", "rssi", "location"]).rename(columns={
    "rssi_x" : "rssi_b",
    "rssi_y" : "rssi_m",
    "rssi" : "rssi_n"
})

In [27]:
merged_data

,rssi_b,rssi_m,rssi_n,location
0,-53,-56,-53,2
1,-66,-63,-60,2
2,-53,-57,-54,2
3,-53,-56,-51,2
4,-54,-57,-53,2
5,-54,-56,-54,2
6,-53,-56,-50,2
7,-66,-63,-59,2
8,-63,-64,-61,2
9,-63,-64,-61,2


In [21]:
grouped_data = merged_data.groupby("location")

In [22]:
def split_to_train_and_test(location):
    data = grouped_data.get_group(location)
    l = len(data)
    train_size = math.floor(3 * l / 4)
    return data[0:train_size], data[train_size:]

In [23]:
train_data = pd.DataFrame()    
test_data = pd.DataFrame()

In [24]:
for loc in range(1, 7):
    print("Adding location {l} to train data".format(l=loc))
    train, test = split_to_train_and_test(loc)
    train_data = pd.concat([train_data, train])
    test_data = pd.concat([test_data, test])
    
train_data = train_data.reset_index()
test_data = test_data.reset_index()


Adding location 1 to train data
Adding location 2 to train data
Adding location 3 to train data
Adding location 4 to train data
Adding location 5 to train data
Adding location 6 to train data


In [25]:
train_data.groupby("location").mean()

,index,rssi_b,rssi_m,rssi_n
location,,,,
1,433.5,-60.298780,-59.670732,-58.283537
2,100.5,-60.108911,-59.688119,-58.079208
3,2066.0,-61.613508,-59.198874,-57.581614
4,1117.0,-59.405372,-59.866911,-57.296703
5,2705.5,-59.507692,-59.887179,-58.938462
6,3307.5,-57.583333,-58.170290,-60.179348


In [226]:
def input_fn_train():
    features = train_data.filter(["rssi_m", "rssi_n", "rssi_b"]).values
    labels = train_data["location"].values
    return features, labels

In [235]:
test_data.filter(["rssi_m", "rssi_n", "rssi_b"])

,rssi_m,rssi_n,rssi_b
0,-70,-48,-60
1,-70,-50,-62
2,-64,-52,-62
3,-65,-60,-61
4,-65,-52,-61
5,-65,-53,-61
6,-65,-62,-61
7,-65,-52,-62
8,-69,-50,-60
9,-68,-49,-60


In [227]:
def input_fn_eval():
    features = test_data.filter(["rssi_m", "rssi_n", "rssi_b"]).values
    labels = test_data["location"].values
    return features, labels

In [228]:
knn = KNeighborsClassifier(n_neighbors=11, leaf_size=50)
x, y = input_fn_train()
knn.fit(x, y)

KNeighborsClassifier(algorithm='auto', leaf_size=50, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=11, p=2,
                     weights='uniform')

In [229]:
test_x, test_y = input_fn_eval()
knn.score(test_x, test_y)

0.43106796116504853

In [230]:
predicted = pd.DataFrame(list(zip(knn.predict(test_x), test_y)))

In [232]:
def get_loc_precision(loc):
    return len(predicted[predicted[1] == loc][predicted[0] == loc]) / len(predicted[predicted[1] == loc])

print(get_loc_precision(1))
print(get_loc_precision(2))
print(get_loc_precision(3))
print(get_loc_precision(4))
print(get_loc_precision(5))


0.3333333333333333
0.24210526315789474
0.4305555555555556
0.27586206896551724
0.6086956521739131


/home/karacasoft/.local/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [136]:
predicted[predicted[1] == 4]

,0,1
221,4,4
222,4,4
223,4,4
224,4,4
225,4,4
226,3,4
227,2,4
228,3,4
229,2,4
230,5,4


In [234]:
import pickle
with open("model.pickle", "wb") as f:
    pickle.dump(knn, f)